In [1]:
%load_ext autoreload
%autoreload 2
from models import baseline_mlp
import numpy as np
import torch
from sklearn.metrics import roc_auc_score

In [12]:
GLOVE_DIR = '/Users/ianmagnusson/IITUDND/data/extracted_features/combined_NLP/irma/'
TFIDF_DIR = '/Users/ianmagnusson/IITUDND/data/extracted_features/combined_NLP/irma/'

In [3]:
def test_100(X_train, y_train, X_test, y_test):
    
    running_acc = 0.0
    running_auc = 0.0
    
    for i in range(100):
        # make model
        baseline = baseline_mlp.MLP(X_train.shape[1], 128)
        
        # train
        baseline.learn(torch.Tensor(X_train), torch.Tensor(y_train))
        
        # evaluate
        running_acc += baseline.get_accuracy(torch.Tensor(X_test), torch.Tensor(y_test))
        running_auc += baseline.get_auc(torch.Tensor(X_test), torch.Tensor(y_test))
    
    return running_acc/100, running_auc/100

def get_auc(model, X_test,y_test):
    with torch.no_grad():
        y_score = model(torch.Tensor(X_test))
    
    y_score = torch.flatten(y_score).numpy()
    return roc_auc_score(y_test, y_score)


In [13]:
# load features
X_labeled_train_glove = np.load(GLOVE_DIR + 'X_labeled_train.npy')
X_histories_train_glove = np.load(GLOVE_DIR + 'X_histories_train.npy')
X_labeled_test_glove = np.load(GLOVE_DIR + 'X_labeled_test.npy')
X_histories_test_glove = np.load(GLOVE_DIR + 'X_histories_test.npy')

y_train_glove = np.load(GLOVE_DIR + 'y_train.npy')
y_test_glove = np.load(GLOVE_DIR + 'y_test.npy')

X_labeled_train_tfidf = np.load(TFIDF_DIR + 'trainTweets.npy')
X_histories_train_tfidf = np.load(TFIDF_DIR + 'trainHistories.npy')
X_labeled_test_tfidf = np.load(TFIDF_DIR + 'testTweets.npy')
X_histories_test_tfidf = np.load(TFIDF_DIR + 'testHistories.npy')

y_train_tfidf = np.load(TFIDF_DIR + 'trainClassifications.npy')
y_test_tfidf = np.load(TFIDF_DIR + 'testClassifications.npy')

In [5]:
# check that the class labels aren't messed up
print(np.array_equal(y_test_glove, y_test_tfidf))
print(np.array_equal(y_train_glove, y_train_tfidf))

True
True


# Unnormalized, full data
 

In [6]:
# merge data

X_train = np.concatenate((X_labeled_train_glove, X_labeled_train_tfidf, X_histories_train_glove, X_histories_train_tfidf), axis = 1)
X_test = np.concatenate((X_labeled_test_glove, X_labeled_test_tfidf, X_histories_test_glove, X_histories_test_tfidf), axis = 1)

y_train = y_train_glove
y_test = y_test_glove


In [15]:
%%capture
# mutes output for this cell

acc, auc = test_100(X_train,y_train,X_test,y_test)

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #4 'mat1'

In [16]:
print(acc)
print(auc)


NameError: name 'acc' is not defined

# Unnormalized, without user histories

In [9]:
# merge data

X_train = np.concatenate((X_labeled_train_glove, X_labeled_train_tfidf), axis = 1)
X_test = np.concatenate((X_labeled_test_glove, X_labeled_test_tfidf), axis = 1)

y_train = y_train_glove
y_test = y_test_glove


In [19]:
%%capture
# mutes output for this cell

acc, auc = test_100(X_train,y_train,X_test,y_test)

In [20]:
print(acc)
print(auc)


0.815420792079208
0.762807916780066


# Normalized, full data

In [14]:
from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
scaler2 = StandardScaler()
scaler3 = StandardScaler()
scaler4 = StandardScaler()


# TODO FIX NAMES!           
X_labeled_train_glove_norm    = scaler1.fit_transform(X_labeled_train_glove)
X_histories_train_glove_norm  = scaler2.fit_transform(X_histories_train_glove)
X_labeled_test_glove_norm     = scaler1.transform(X_labeled_test_glove)
X_histories_test_glove_norm  = scaler2.transform(X_histories_test_glove)

X_labeled_train_tfidf_norm    = scaler3.fit_transform(X_labeled_train_tfidf)
X_histories_train_tfidf_norm  = scaler4.fit_transform(X_histories_train_tfidf)
X_labeled_test_tfidf_norm     = scaler3.transform(X_labeled_test_tfidf)
X_histories_test_tfidf_norm   = scaler4.transform(X_histories_test_tfidf)



In [15]:
# merge data

X_train = np.concatenate((X_labeled_train_glove_norm, X_labeled_train_tfidf_norm,
                          X_histories_train_glove_norm, X_histories_train_tfidf_norm),
                         axis = 1)
X_test = np.concatenate((X_labeled_test_glove_norm, X_labeled_test_tfidf_norm,
                         X_histories_test_glove_norm, X_histories_test_tfidf_norm),
                        axis = 1)

y_train = y_train_glove
y_test = y_test_glove


In [16]:
%%capture
# mutes output for this cell

acc, auc = test_100(X_train,y_train,X_test,y_test)

In [17]:
print(acc)
print(auc)

0.8160767326732679
0.7633535940004235


# Normalized, without histories

In [18]:
# merge data

X_train = np.concatenate((X_labeled_train_glove_norm, X_labeled_train_tfidf_norm),
                         axis = 1)
X_test = np.concatenate((X_labeled_test_glove_norm, X_labeled_test_tfidf_norm),
                        axis = 1)

y_train = y_train_glove
y_test = y_test_glove

In [19]:
%%capture
# mutes output for this cell

acc, auc = test_100(X_train,y_train,X_test,y_test)

In [20]:
print(acc)
print(auc)


0.819789603960396
0.7680467104135795


# MISC TESTS

# Now with just user histories


In [23]:
# merge data

X_train = np.concatenate(( X_histories_train_glove, X_histories_train_tfidf), axis = 1)
X_test = np.concatenate(( X_histories_test_glove, X_histories_test_tfidf), axis = 1)

y_train = y_train_glove
y_test = y_test_glove


In [24]:
# make model
baseline = baseline_mlp.MLP(X_train.shape[1], 400, 200, torch.relu)

# train
baseline.learn(torch.Tensor(X_train), torch.Tensor(y_train))

# evaluate
acc0 = baseline.get_accuracy(torch.Tensor(X_test), torch.Tensor(y_test))
print('accuracy:', acc0)


epoch: 0 learning rate: [0.1]
[1,   200] loss: 0.493
[1,   400] loss: 0.482
[1,   600] loss: 0.448
[1,   800] loss: 0.438
epoch: 1 learning rate: [0.010000000000000002]
[2,   200] loss: 0.390
[2,   400] loss: 0.405
[2,   600] loss: 0.383
[2,   800] loss: 0.428
accuracy: 0.8081683168316832


# Now with just tfidf

In [25]:
# merge data

X_train = np.concatenate((X_labeled_train_tfidf, X_histories_train_tfidf), axis = 1)
X_test = np.concatenate((X_labeled_test_tfidf, X_histories_test_tfidf), axis = 1)

y_train = y_train_glove
y_test = y_test_glove


In [26]:
# make model
baseline = baseline_mlp.MLP(X_train.shape[1], 400, 200, torch.relu)

# train
baseline.learn(torch.Tensor(X_train), torch.Tensor(y_train))

# evaluate
acc0 = baseline.get_accuracy(torch.Tensor(X_test), torch.Tensor(y_test))
print('accuracy:', acc0)

epoch: 0 learning rate: [0.1]
[1,   200] loss: 0.533
[1,   400] loss: 0.500
[1,   600] loss: 0.509
[1,   800] loss: 0.528
epoch: 1 learning rate: [0.010000000000000002]
[2,   200] loss: 0.514
[2,   400] loss: 0.475
[2,   600] loss: 0.486
[2,   800] loss: 0.496
accuracy: 0.8131188118811881


# Now with just glove

In [27]:
# merge data

X_train = np.concatenate((X_labeled_train_glove, X_histories_train_glove ), axis = 1)
X_test = np.concatenate((X_labeled_test_glove, X_histories_test_glove), axis = 1)

y_train = y_train_glove
y_test = y_test_glove


In [28]:
# make model
baseline = baseline_mlp.MLP(X_train.shape[1], 400, 200, torch.relu)

# train
baseline.learn(torch.Tensor(X_train), torch.Tensor(y_train))

# evaluate
acc0 = baseline.get_accuracy(torch.Tensor(X_test), torch.Tensor(y_test))
print('accuracy:', acc0)

epoch: 0 learning rate: [0.1]
[1,   200] loss: 0.501
[1,   400] loss: 0.429
[1,   600] loss: 0.461
[1,   800] loss: 0.435
epoch: 1 learning rate: [0.010000000000000002]
[2,   200] loss: 0.406
[2,   400] loss: 0.386
[2,   600] loss: 0.373
[2,   800] loss: 0.387
accuracy: 0.8193069306930693


# Now glove without histories

In [29]:
# merge data

X_train = X_labeled_train_glove
X_test = X_labeled_test_glove

y_train = y_train_glove
y_test = y_test_glove


In [30]:
# make model
baseline = baseline_mlp.MLP(X_train.shape[1], 200, 200, torch.relu)

# train
baseline.learn(torch.Tensor(X_train), torch.Tensor(y_train))

# evaluate
acc0 = baseline.get_accuracy(torch.Tensor(X_test), torch.Tensor(y_test))
print('accuracy:', acc0)



epoch: 0 learning rate: [0.1]
[1,   200] loss: 0.523
[1,   400] loss: 0.466
[1,   600] loss: 0.411
[1,   800] loss: 0.435
epoch: 1 learning rate: [0.010000000000000002]
[2,   200] loss: 0.398
[2,   400] loss: 0.372
[2,   600] loss: 0.389
[2,   800] loss: 0.410
accuracy: 0.8168316831683168


In [31]:
with torch.no_grad():
    y_score = baseline(torch.Tensor(X_test))

y_score = torch.flatten(y_score).numpy()
roc_auc_score(y_test, y_score)


0.7683227998024332

In [32]:
print(__doc__)
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()



Automatically created module for IPython interactive environment


<Figure size 640x480 with 1 Axes>

# Try normalized (MINMAX)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
scaler3 = MinMaxScaler()
scaler4 = MinMaxScaler()


# HAS BUGS!

            
'''X_labeled_train_glove_minmax    = scaler1.fit_transform(X_labeled_train_glove)
X_histories_train_glove_minmax  = scaler2.fit_transform(X_histories_train_glove)
X_labeled_test_glove_minmax     = scaler3.fit_transform(X_labeled_test_glove)
X_histories_test_glove_minmax   = scaler4.fit_transform(X_histories_test_glove)

X_labeled_train_tfidf_minmax    = scaler1.transform(X_labeled_train_tfidf)
X_histories_train_tfidf_minmax  = scaler2.transform(X_histories_train_tfidf)
X_labeled_test_tfidf_minmax     = scaler3.transform(X_labeled_test_tfidf)
X_histories_test_tfidf_minmax   = scaler4.transform(X_histories_test_tfidf)'''



In [ ]:
# merge data

X_train = np.concatenate((X_labeled_train_glove_minmax, X_labeled_train_tfidf_minmax,
                          X_histories_train_glove_minmax, X_histories_train_tfidf_minmax),
                         axis = 1)
X_test = np.concatenate((X_labeled_test_glove_minmax, X_labeled_test_tfidf_minmax,
                         X_histories_test_glove_minmax, X_histories_test_tfidf_minmax),
                        axis = 1)

y_train = y_train_glove
y_test = y_test_glove


In [ ]:
%%capture
# mutes output for this cell

acc, auc = test_100(X_train,y_train,X_test,y_test)

In [ ]:
print(acc)
print(auc)
